In [ ]:
import pandas as pd
import numpy as np
import json
import os
import requests
import time

# Please do not run any cells past this point until directed otherwise

Many of these cells require API access to function. Please do not run any cells until you see this comment:

"To recreate the data transformation process, please run the cells below."

Additionally, data frames are frequently saved to the file system throughout this code. All file system interaction has been commented out to prevent unintentional overwriting of existing csvs.

# Data Collection

In [ ]:
# Insert API key as environmental variable before running the notebook. Variable is 'key'
#%env key=

In [ ]:
# Importing a pre-made csv file containing all three hundrend summoners in the Challenger league
df = pd.read_csv('challenger_test.csv')
df

In [ ]:
df.info()

In [ ]:
type(df['entries'].iloc[0])

In [ ]:
# Converting JSON data to Python dictionary
df['entries'] = df['entries'].map(eval)

In [ ]:
# Creating a column for each key in the 'entries' column
df2 = pd.concat([df.drop('entries', axis=1), df['entries'].apply(pd.Series)], axis=1)

In [ ]:
df2.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
#df.to_csv('formatted_challenger')

### API Interface Functions

In [ ]:
def get_challenger(key):
    r = requests.get("https://na1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5",
            params={'api_key': key})
    print(r.url)

    return r.text

def get_puuid(summoner_id, key):
    r = requests.get("https://na1.api.riotgames.com/lol/summoner/v4/summoners/" + summoner_id,
                     params={'api_key': key})
    print(r.url)
    
    return r.text

def get_matches(puuid, key):
    r = requests.get("https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/" + puuid + "/ids",
                    params={'start': 0, 'count': 100, 'api_key': key})
    print(r.url)
    
    return r.text

def get_match_stats(match_id, key):
    r = requests.get("https://americas.api.riotgames.com/lol/match/v5/matches/" + match_id,
                     params={'api_key': key})
    print(r.url)
    
    return r.text

### Pulling summoner account data from the API

In [ ]:
testdf = pd.DataFrame([])
testdf

In [ ]:
# Pulling summoner data from the API. If statement is necessary due to rate limiting
for i in range(len(df2)):
    if i != 0 and i % 100 == 0:
        time.sleep(120)
    testdf = pd.concat([testdf, pd.Series(get_puuid(df2['summonerId'][i], os.getenv('key')))], ignore_index=True)
    
testdf

In [ ]:
testdf.info()

In [ ]:
#testdf.to_csv('unformatted_summoner_data')

In [ ]:
# Formatting data to convert into Python dictionary
testdf[0] = testdf[0].apply(lambda x: x.replace('\"', "\'"))

In [ ]:
testdf[0] = testdf[0].map(eval)

In [ ]:
# Creating a new formatted DataFrame containing summoner PUUIDs from formatted data
puuid_df = pd.concat([testdf.drop(0, axis=1), testdf[0].apply(pd.Series)], axis=1)

In [ ]:
puuid_df

In [ ]:
# Checking for empty rows
puuid_empty = puuid_df[puuid_df.isna().any(axis=1)]

In [ ]:
puuid_empty

In [ ]:
# Removing identified empty row
puuid_df.drop(76, axis=0, inplace=True)

In [ ]:
puuid_df = puuid_df.reset_index()

In [ ]:
puuid_df.drop('index', axis=1, inplace=True)
puuid_df

In [ ]:
#puuid_df.to_csv('formatted_summoner_data')

### Pulling list of matches for each summoner PUUID from the API

In [ ]:
unformatted_match_df = pd.DataFrame([])
unformatted_match_df

In [ ]:
# If statement necessary to account for rate limiting
for i in range(len(puuid_df)):
    if i != 0 and i % 100 == 0:
        time.sleep(120)
    unformatted_match_df = pd.concat([unformatted_match_df, pd.Series(
                            get_matches(puuid_df['puuid'][i], os.getenv('key')))],
                            ignore_index=True)

In [ ]:
# Reformatting to simplify DataFrame into a single column
formatted_match_df = pd.DataFrame()
for i in range(len(unformatted_match_df)):
    temp = unformatted_match_df[i]
    formatted_match_df = pd.concat([formatted_match_df, pd.DataFrame(temp)], ignore_index=True)

formatted_match_df

In [ ]:
# Removing duplicate rows
formatted_match_df = formatted_match_df.drop_duplicates()

In [ ]:
formatted_match_df

In [ ]:
formatted_match_df = formatted_match_df.reset_index()

In [ ]:
formatted_match_df = formatted_match_df.drop('index', axis=1)
formatted_match_df

In [ ]:
#formatted_match_df.to_csv('formatted_match_df.csv')

### Pulling match data for each match in the formatted_match_df

In [ ]:
formatted_match_stats_df = pd.DataFrame()

for i in range(len(formatted_match_df)):
    if i != 0 and i % 100 == 0:
        print(i)
        time.sleep(120)
    temp = pd.read_json(get_match_stats(formatted_match_df[0][i], os.getenv('key')),
                        orient='index').drop('metadata', axis=0).drop(['dataVersion', 'matchId'], axis=1)
    temp = pd.concat([temp.drop(['participants', 'teams'], axis=1), temp['participants'].apply(pd.Series).rename(columns={
                                                        0:'T1P1',1:'T1P2',2:'T1P3',3:'T1P4',4:'T1P5',
                                                        5:'T2P1',6:'T2P2',7:'T2P3',8:'T2P4',9:'T2P5'}),
                                                        temp['teams'].apply(pd.Series).rename(columns={
                                                        0:'teamOne', 1:'teamTwo'})], axis=1)
    
    formatted_match_stats_df = pd.concat([formatted_match_stats_df, temp], axis=0, ignore_index=True)

In [ ]:
formatted_match_stats_df

# To recreate the data transformation process, please run the cells below

In [ ]:
#formatted_match_stats_df.to_csv('formatted_match_stats_df.csv')
formatted_match_stats_df = pd.read_csv('formatted_match_stats_df.csv')

In [ ]:
# Removing non-competitive matches
formatted_match_stats_df = formatted_match_stats_df[formatted_match_stats_df.gameType != "CUSTOM_GAME"]

In [ ]:
# Removing more non-competitive matches
formatted_match_stats_df = formatted_match_stats_df[(formatted_match_stats_df.gameMode != 'ARAM') & 
                         (formatted_match_stats_df.gameMode != 'TUTORIAL_MODULE_1') &
                         (formatted_match_stats_df.gameMode != 'ONEFORALL')]

In [ ]:
formatted_match_stats_df

In [ ]:
formatted_match_stats_df.gameMode.unique()

In [ ]:
formatted_match_stats_df.info()

In [ ]:
formatted_match_stats_df.drop('Unnamed: 0', axis=1, inplace=True)
formatted_match_stats_df

In [ ]:
formatted_match_stats_df.drop(['gameCreation', 'gameDuration', 'gameEndTimestamp', 'gameId', 'gameMode', 'gameStartTimestamp',
                               'gameType', 'gameVersion', 'mapId', 'gameName', 'platformId', 
                               'queueId', 'tournamentCode'], axis=1, inplace=True)

In [ ]:
formatted_match_stats_df

In [ ]:
#formatted_match_stats_df.to_csv('probably_final_stats.csv')

In [ ]:
# Formatting match status to ensure all columns are numeric and contain a single feature
needed_keys = ['totalTimeSpentDead', 'goldEarned', 
               'wardsKilled', 'totalMinionsKilled', 'wardsPlaced']

In [ ]:
# Testing dictionary comprehension that will be used for formatting
player_dict = {key:value for key,value in eval(formatted_match_stats_df['T1P1'][0]).items() if key in needed_keys}
player_dict

In [ ]:
# Converting team one player one column to Python dictionary
formatted_match_stats_df['T1P1'] = formatted_match_stats_df['T1P1'].map(eval)
formatted_match_stats_df['T1P1']

In [ ]:
# Converting team one player two column to Python dictionary
formatted_match_stats_df['T1P2'] = formatted_match_stats_df['T1P2'].map(eval)
formatted_match_stats_df['T1P2']

In [ ]:
# Removing unwanted features in team one player one column
formatted_match_stats_df['T1P1'] = \
    formatted_match_stats_df['T1P1'].apply(lambda x: {key:value for key,value in x.items() if key in needed_keys})

In [ ]:
formatted_match_stats_df['T1P1']

In [ ]:
# Removing unwanted features in team one player two column
formatted_match_stats_df['T1P2'] = \
    formatted_match_stats_df['T1P2'].apply(lambda x: {key:value for key,value in x.items() if key in needed_keys})

In [ ]:
# Converting remaining player columns to Python dictionaries
formatted_match_stats_df['T1P3'] = formatted_match_stats_df['T1P3'].map(eval)

formatted_match_stats_df['T1P4'] = formatted_match_stats_df['T1P4'].map(eval)

formatted_match_stats_df['T1P5'] = formatted_match_stats_df['T1P5'].map(eval)

formatted_match_stats_df['T2P1'] = formatted_match_stats_df['T2P1'].map(eval)

formatted_match_stats_df['T2P2'] = formatted_match_stats_df['T2P2'].map(eval)

formatted_match_stats_df['T2P3'] = formatted_match_stats_df['T2P3'].map(eval)

formatted_match_stats_df['T2P4'] = formatted_match_stats_df['T2P4'].map(eval)

formatted_match_stats_df['T2P5'] = formatted_match_stats_df['T2P5'].map(eval)

In [ ]:
# Removing unwanted features in remaining player columns
formatted_match_stats_df['T1P3'] = \
    formatted_match_stats_df['T1P3'].apply(lambda x: {key:value for key,value in x.items() if key in needed_keys})

formatted_match_stats_df['T1P4'] = \
    formatted_match_stats_df['T1P4'].apply(lambda x: {key:value for key,value in x.items() if key in needed_keys})

formatted_match_stats_df['T1P5'] = \
    formatted_match_stats_df['T1P5'].apply(lambda x: {key:value for key,value in x.items() if key in needed_keys})

formatted_match_stats_df['T2P1'] = \
    formatted_match_stats_df['T2P1'].apply(lambda x: {key:value for key,value in x.items() if key in needed_keys})

formatted_match_stats_df['T2P2'] = \
    formatted_match_stats_df['T2P2'].apply(lambda x: {key:value for key,value in x.items() if key in needed_keys})

formatted_match_stats_df['T2P3'] = \
    formatted_match_stats_df['T2P3'].apply(lambda x: {key:value for key,value in x.items() if key in needed_keys})

formatted_match_stats_df['T2P4'] = \
    formatted_match_stats_df['T2P4'].apply(lambda x: {key:value for key,value in x.items() if key in needed_keys})

formatted_match_stats_df['T2P5'] = \
    formatted_match_stats_df['T2P5'].apply(lambda x: {key:value for key,value in x.items() if key in needed_keys})

In [ ]:
formatted_match_stats_df

In [ ]:
# Expanding each team one player column to ensure each column contains only one feature
match_stats_test_df = pd.concat([formatted_match_stats_df.drop(['T1P1', 'T1P2', 'T1P3', 'T1P4', 'T1P5'], axis=1), 
                                       formatted_match_stats_df['T1P1'].apply(pd.Series),
                                       formatted_match_stats_df['T1P2'].apply(pd.Series),
                                       formatted_match_stats_df['T1P3'].apply(pd.Series),
                                       formatted_match_stats_df['T1P4'].apply(pd.Series),
                                       formatted_match_stats_df['T1P5'].apply(pd.Series)],
                                      axis=1)

In [ ]:
match_stats_test_df

In [ ]:
# Grouping by column name to aggregate all player stats for team one
match_stats_test_df = match_stats_test_df.groupby(match_stats_test_df.columns, axis=1).sum()

In [ ]:
match_stats_test_df

In [ ]:
# Renaming columns to properly identify team one columns
match_stats_test_df.rename(columns={'goldEarned':'T1goldEarned', 'totalMinionsKilled':'T1totalMinionsKilled',
                                    'totalTimeSpentDead':'T1totalTimeSpentDead', 'wardsKilled':'T1wardsKilled',
                                    'wardsPlaced':'T1wardsPlaced'}, inplace=True)

In [ ]:
match_stats_test_df

In [ ]:
# Expanding each team two player column to ensure each column contains only one feature
match_stats_test_df = pd.concat([match_stats_test_df.drop(['T2P1', 'T2P2', 'T2P3', 'T2P4', 'T2P5'], axis=1), 
                                       match_stats_test_df['T2P1'].apply(pd.Series),
                                       match_stats_test_df['T2P2'].apply(pd.Series),
                                       match_stats_test_df['T2P3'].apply(pd.Series),
                                       match_stats_test_df['T2P4'].apply(pd.Series),
                                       match_stats_test_df['T2P5'].apply(pd.Series)],
                                      axis=1)

In [ ]:
match_stats_test_df

In [ ]:
# Grouping by column name to aggregate all player stats for team two
match_stats_test_df = match_stats_test_df.groupby(match_stats_test_df.columns, axis=1).sum()

In [ ]:
# Renaming columns to properly identify team two columns
match_stats_test_df.rename(columns={'goldEarned':'T2goldEarned', 'totalMinionsKilled':'T2totalMinionsKilled',
                                    'totalTimeSpentDead':'T2totalTimeSpentDead', 'wardsKilled':'T2wardsKilled',
                                    'wardsPlaced':'T2wardsPlaced'}, inplace=True)

In [ ]:
# Formatting more matchs stat columns to ensure all columns are numeric and contain a single feature
needed_keys = ['objectives', 'win']

In [ ]:
# Converting columns to dictionaries
match_stats_test_df['teamOne'] = match_stats_test_df['teamOne'].map(eval)
match_stats_test_df['teamTwo'] = match_stats_test_df['teamTwo'].map(eval)

In [ ]:
# Reducing features
match_stats_test_df['teamOne'] = \
    match_stats_test_df['teamOne'].apply(lambda x: {key:value for key,value in x.items() if key in needed_keys})

In [ ]:
# Reducing features
needed_keys = ['objectives']
match_stats_test_df['teamTwo'] = \
    match_stats_test_df['teamTwo'].apply(lambda x: {key:value for key,value in x.items() if key in needed_keys})

In [ ]:
match_stats_test_df['teamOne'][0]

In [ ]:
match_stats_test_df['teamTwo'][0]

In [ ]:
# Expanding columns to ensure each column only has one feature
match_stats_test_formatted_df = pd.concat([match_stats_test_df.drop(['teamOne', 'teamTwo'], axis=1), 
                                       match_stats_test_df['teamOne'].apply(pd.Series),
                                       match_stats_test_df['teamTwo'].apply(pd.Series),],
                                      axis=1)

In [ ]:
match_stats_test_formatted_df

In [ ]:
# Appropriately renaming the team one win column created through expansion
match_stats_test_formatted_df.rename(columns={'win':'T1win'}, inplace=True)

In [ ]:
match_stats_test_formatted_df

In [ ]:
# Remaining columns to remove duplicate column names
match_stats_test_formatted_df.columns = ['T1goldEarned', 'T1totalMinionsKilled', 'T1totalTimeSpentDead', 'T1wardsKilled',
                                         'T1wardsPlaced', 'T2goldEarned', 'T2totalMinionsKilled', 'T2totalTimeSpentDead',
                                         'T2wardsKilled', 'T2wardsPlaced', 'T1objectives', 'T1win', 'T2objectives']

In [ ]:
match_stats_test_formatted_df

In [ ]:
# Ensuring that T1/T2/objectives columns contains Python dictionaries, may get an error.
# If so, move on to the next cell.
match_stats_test_formatted_df['T1objectives'] = match_stats_test_formatted_df['T1objectives'].map(eval)
match_stats_test_formatted_df['T2objectives'] = match_stats_test_formatted_df['T2objectives'].map(eval)

In [ ]:
# Writing a function to reduce features in objective columns
def remove_key(objective_dict):
    objective_dict.get('baron').pop('first')
    objective_dict.get('champion').pop('first')
    objective_dict.get('dragon').pop('first')
    objective_dict.get('inhibitor').pop('first')
    objective_dict.get('riftHerald').pop('first')
    objective_dict.get('tower').pop('first')
    return

In [ ]:
match_stats_test_formatted_df['T1objectives'].map(remove_key)

In [ ]:
match_stats_test_formatted_df

In [ ]:
# Expanding T1objecrtives into multiple columns
match_stats_test_formatted_df = pd.concat([match_stats_test_formatted_df.drop('T1objectives', axis=1),
                                           match_stats_test_formatted_df['T1objectives'].apply(pd.Series)],
                                         axis=1)

In [ ]:
match_stats_test_formatted_df

In [ ]:
# Function to transform objective columns' data into numeric form
def get_kills(kill_dict):
    return kill_dict.get('kills')

In [ ]:
# Transforming objective columns' data into numberic form
match_stats_test_formatted_df['baron'] = match_stats_test_formatted_df['baron'].apply(get_kills)

match_stats_test_formatted_df['champion'] = match_stats_test_formatted_df['champion'].apply(get_kills)

match_stats_test_formatted_df['dragon'] = match_stats_test_formatted_df['dragon'].apply(get_kills)

match_stats_test_formatted_df['inhibitor'] = match_stats_test_formatted_df['inhibitor'].apply(get_kills)

match_stats_test_formatted_df['riftHerald'] = match_stats_test_formatted_df['riftHerald'].apply(get_kills)

match_stats_test_formatted_df['tower'] = match_stats_test_formatted_df['tower'].apply(get_kills)

In [ ]:
match_stats_test_formatted_df

In [ ]:
# Renaming columns to identify them as belong to team one
match_stats_test_formatted_df.rename(columns={"baron": "T1baron", "champion": "T1champion", "dragon":"T1dragon",
                                               "inhibitor":"T1inhibitor", "riftHerald":"T1riftHerald", "tower":"T1tower"},
                                     inplace=True)

In [ ]:
match_stats_test_formatted_df

In [ ]:
# Repeating previous steps for T2objectives
match_stats_test_formatted_df['T2objectives'].map(remove_key)

match_stats_test_formatted_df = pd.concat([match_stats_test_formatted_df.drop('T2objectives', axis=1),
                                           match_stats_test_formatted_df['T2objectives'].apply(pd.Series)],
                                         axis=1)

match_stats_test_formatted_df

In [ ]:
match_stats_test_formatted_df['baron'] = match_stats_test_formatted_df['baron'].apply(get_kills)

match_stats_test_formatted_df['champion'] = match_stats_test_formatted_df['champion'].apply(get_kills)

match_stats_test_formatted_df['dragon'] = match_stats_test_formatted_df['dragon'].apply(get_kills)

match_stats_test_formatted_df['inhibitor'] = match_stats_test_formatted_df['inhibitor'].apply(get_kills)

match_stats_test_formatted_df['riftHerald'] = match_stats_test_formatted_df['riftHerald'].apply(get_kills)

match_stats_test_formatted_df['tower'] = match_stats_test_formatted_df['tower'].apply(get_kills)

In [ ]:
match_stats_test_formatted_df

In [ ]:
match_stats_test_formatted_df.rename(columns={"baron": "T2baron", "champion": "T2champion", "dragon":"T2dragon",
                                               "inhibitor":"T2inhibitor", "riftHerald":"T2riftHerald", "tower":"T2tower"},
                                     inplace=True)

In [ ]:
match_stats_test_formatted_df

In [ ]:
# Transforming boolean data to integer form
match_stats_test_formatted_df['T1win'] = match_stats_test_formatted_df['T1win'].astype(int)

In [ ]:
#match_stats_test_formatted_df.to_csv('fully_numeric_match_stats.csv')